In [ ]:
import numpy as np
import tifffile as tiff
import nanopyx.liquid as npx_cl
from matplotlib import pyplot as plt

In [ ]:
task_run_times_unthreaded = []
task_run_times_threaded = []
task_run_times_threaded_static = []
task_run_times_threaded_guided = []
task_run_times_threaded_dynamic = []
task_run_times_opencl = []

In [ ]:
conv2d = npx_cl.Convolution2D()
dim_sizes  = [(dim, dim) for dim in [100, 500, 1000, 2500, 5000, 7500, 10000, 15000, 20000]]
kernel_sizes = [(dim, dim) for dim in range(1, 25, 4)]

In [ ]:
for dim in dim_sizes:
    img = np.random.random(dim).astype(np.float32)
    for ks in kernel_sizes:
        kernel = np.ones(ks).astype(np.float32)
        conv2d.run(img, kernel,run_type="Threaded")
        task_run_times_threaded.append((dim[0], ks[0], conv2d._last_time))
        a = conv2d._last_time
        #print("Threaded finished")

        conv2d.run(img,kernel,run_type="Threaded_static")
        task_run_times_threaded_static.append((dim[0], ks[0], conv2d._last_time))

        conv2d.run(img,kernel,run_type="Threaded_guided")
        task_run_times_threaded_guided.append((dim[0], ks[0], conv2d._last_time))

        conv2d.run(img,kernel,run_type="Threaded_dynamic")
        task_run_times_threaded_dynamic.append((dim[0], ks[0], conv2d._last_time))

        conv2d.run(img,kernel,run_type="Unthreaded")
        task_run_times_unthreaded.append((dim[0], ks[0], conv2d._last_time))
        #print("UnThreaded finished")

        conv2d.run(img,kernel,run_type="OpenCL_Apple M1") # change to appropriate device name (OpenCL_Apple M1 Pro, OpenCL_NVIDIA GeForce RTX 4090)
        task_run_times_opencl.append((dim[0], ks[0], conv2d._last_time))
        #print("Threaded finished")
        
        b = conv2d._last_time
    
        print(b/a)

In [ ]:
# Run this cell to store benchmark values for future usage

task_run_times_unthreaded = np.array(task_run_times_unthreaded)
task_run_times_threaded = np.array(task_run_times_threaded)
task_run_times_threaded_static = np.array(task_run_times_threaded_static)
task_run_times_threaded_guided = np.array(task_run_times_threaded_guided)
task_run_times_threaded_dynamic = np.array(task_run_times_threaded_dynamic)
task_run_times_opencl = np.array(task_run_times_opencl)

np.save("task_run_times_opencl.npy", task_run_times_opencl)
np.save("task_run_times_unthreaded.npy", task_run_times_unthreaded)
np.save("task_run_times_threaded.npy", task_run_times_threaded)
np.save("task_run_times_threaded_static.npy", task_run_times_threaded_static)
np.save("task_run_times_threaded_guided.npy", task_run_times_threaded_guided)
np.save("task_run_times_threaded_dynamic.npy", task_run_times_threaded_dynamic)

In [ ]:
# Run this cell if you're importing previous benchmarks

import os
import numpy as np

fld = "M1"
task_run_times_opencl = np.load("Runtimes" + fld + os.sep + "task_run_times_opencl.npy")
task_run_times_unthreaded = np.load("Runtimes" + fld + os.sep + "task_run_times_unthreaded.npy")
task_run_times_threaded = np.load("Runtimes" + fld + os.sep + "task_run_times_threaded.npy")
task_run_times_threaded_static = np.load("Runtimes" + fld + os.sep + "task_run_times_threaded_static.npy")
task_run_times_threaded_guided = np.load("Runtimes" + fld + os.sep + "task_run_times_threaded_guided.npy")
task_run_times_threaded_dynamic = np.load("Runtimes" + fld + os.sep + "task_run_times_threaded_dynamic.npy")

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib.colors import LogNorm

sns.color_palette("Spectral", as_cmap=True)

ratio_threaded_opencl = task_run_times_opencl[:, 2]/task_run_times_threaded[:, 2]
ratio_threaded_opencl = np.array(ratio_threaded_opencl).reshape(9, 6)
ax = sns.heatmap(ratio_threaded_opencl, annot=True, cmap="Spectral",
                 xticklabels=[k[0] for k in kernel_sizes],
                 yticklabels=[4*i[0]**2 / 1000000 for i in dim_sizes],
                 norm=LogNorm(),
                 fmt=".1f")
plt.title("Ratio between OpenCL and Threaded CPU run times")
plt.ylabel("Input image size (MB)")
plt.xlabel("Kernel size (Size x Size)")
plt.savefig("/Users/bsaraiva/Code/NanoPyx/ratio_threaded_opencl_" + fld + ".pdf", dpi=300)
plt.show()

In [ ]:
# plot per data shape
index = 5
step = 6
image_sizes = [4*i[0]**2 / 1000000 for i in dim_sizes]
plt.figure(figsize=(6, 5))
plt.plot(image_sizes, task_run_times_unthreaded[index::step, 2], 'orange', label='Cython unthreaded')
plt.plot(image_sizes, task_run_times_threaded[index::step, 2], 'blue', label='Cython threaded')
plt.plot(image_sizes, task_run_times_threaded_static[index::step, 2], 'lightpink', label='Cython threaded static')
plt.plot(image_sizes, task_run_times_threaded_guided[index::step, 2], 'green', label='Cython threaded guided')
plt.plot(image_sizes, task_run_times_threaded_dynamic[index::step, 2], 'black', label='Cython threaded dynamic')
plt.plot(image_sizes, task_run_times_opencl[index::step, 2], 'magenta', label='pyOpenCL')

In [ ]:
# plot per kernel shape
index = 3
step = 6
plt.figure(figsize=(6, 5))
plt.plot(task_run_times_unthreaded[index*step:(index*step)+step, 1], task_run_times_unthreaded[index*step:(index*step)+step, 2], 'orange', label='Cython unthreaded')
plt.plot(task_run_times_threaded[index*step:(index*step)+step, 1], task_run_times_threaded[index*step:(index*step)+step, 2], 'blue', label='Cython threaded')
plt.plot(task_run_times_threaded_static[index*step:(index*step)+step, 1], task_run_times_threaded_static[index*step:(index*step)+step, 2], 'lightpink', label='Cython threaded static')
plt.plot(task_run_times_threaded_guided[index*step:(index*step)+step, 1], task_run_times_threaded_guided[index*step:(index*step)+step, 2], 'green', label='Cython threaded guided')
plt.plot(task_run_times_threaded_dynamic[index*step:(index*step)+step, 1], task_run_times_threaded_dynamic[index*step:(index*step)+step, 2], 'black', label='Cython threaded dynamic')
plt.plot(task_run_times_opencl[index*step:(index*step)+step, 1], task_run_times_opencl[index*step:(index*step)+step, 2], 'magenta', label='pyOpenCL')